In [1]:
import pandas as pd
import numpy as np
import psycopg2

db_navn = 'AC Horsens'
db_brugernavn = 'postgres'
db_adgangskode = 'ACHorsens'
db_host = 'localhost'
conn = psycopg2.connect(
    dbname=db_navn,
    user=db_brugernavn,
    password=db_adgangskode,
    host=db_host
)

cur = conn.cursor()
cur.execute("SELECT schema_name FROM information_schema.schemata")
schemas = cur.fetchall()
print('connected to database')
def map_to_unified_columns_possession(df):
    # Create a dictionary mapping the original column names to the unified column names
    column_mapping = {
        'contestantId': 'contestantId',
        'team_name': 'team_name',
        'id': 'id',
        'eventId': 'eventId',
        'typeId': 'typeId',
        'periodId': 'periodId',
        'timeMin': 'timeMin',
        'timeSec': 'timeSec',
        'outcome': 'outcome',
        'x': 'x',
        'y': 'y',
        'timeStamp': 'timeStamp',
        'lastModified': 'lastModified',
        'playerId': 'playerId',
        'playerName': 'playerName',
        'sequenceId': 'sequenceId',
        'possessionId': 'possessionId',
        'keyPass': 'keyPass',
        'assist': 'assist',
        '140.0': '140.0',
        '141.0': '141.0',
        '318.0': '318.0',
        '321.0': '321.0',
        '210.0': '210',
        '22.0': '22.0',
        '23.0': '23.0',
        '5.0' : '5.0',
        '6.0' : '6.0',
        '9.0' : '9.0',
        '24.0': '24.0',
        '25.0': '25.0',
        '26.0': '26.0',
        '107.0':'107.0',
        '210.0':'210.0',
        '213.0':'213.0',
        'match_id': 'match_id',
        'label': 'label',
        'date': 'date',
    }
    for col in column_mapping.values():
        if col not in df.columns:
            df[col] = None

    # Rename columns using the mapping dictionary
    unified_df = df.rename(columns=column_mapping)

    return unified_df

from concurrent.futures import ThreadPoolExecutor
import psycopg2
import pandas as pd
import io
from psycopg2 import sql

def fast_table_to_df(cur, schema, table):
    query = sql.SQL('COPY {}.{} TO STDOUT WITH CSV HEADER').format(
        sql.Identifier(schema),
        sql.Identifier(table)
    )
    buffer = io.StringIO()
    cur.copy_expert(query, buffer)
    buffer.seek(0)
    return pd.read_csv(buffer)

def process_schema(schema):
    local_conn = psycopg2.connect(
        dbname=db_navn,
        user=db_brugernavn,
        password=db_adgangskode,
        host=db_host,
        port="5432"
    )
    local_cur = local_conn.cursor()

    result_dfs = []
    try:
        local_cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s", (schema,))
        tables = [t[0] for t in local_cur.fetchall() if 'possession_data' in t[0]]

        for table in tables:
            print(f"Processing {schema}.{table}")
            df = fast_table_to_df(local_cur, schema, table)
            df = map_to_unified_columns_possession(df)
            df = df[['timeMin', 'timeSec', 'x', 'y','team_name', 'playerName', 'label','date','typeId','sequence_xG','321.0', '322.0','receiverName']]
            result_dfs.append(df)
    except Exception as e:
        print(f"Error processing {schema}: {e}")
    finally:
        local_cur.close()
        local_conn.close()

    return result_dfs

schemas_to_include = [
    'DNK_1_Division_2023_2024',
    'DNK_1_Division_2024_2025',
    'DNK_1_Division_2025_2026'
    'SRB_Super_Liga_2024_2025',
    'SRB_Super_Liga_2025_2026',
    #'DNK_Superliga_2024_2025'
]

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_schema, schemas_to_include))

# Flatten and combine
all_data_frames = [df for dfs in results for df in dfs]
combined_df = pd.concat(all_data_frames, ignore_index=True)
combined_df.to_csv('Striker reports.csv')

connected to database
Processing DNK_1_Division_2023_2024.Hobro_vs_AaB2023_08_24possession_data
Processing DNK_1_Division_2023_2024.HB_Køge_vs_AaB2023_12_03possession_data
Processing DNK_1_Division_2023_2024.Hobro_vs_HB_Køge2023_10_27possession_data
Processing DNK_1_Division_2023_2024.Horsens_vs_Næstved2023_08_25possession_data
Processing DNK_1_Division_2024_2025.Kolding_vs_HB_Køge2024_09_14possession_data
Processing DNK_1_Division_2024_2025.Hvidovre_vs_Fredericia2025_02_27possession_data
Processing DNK_1_Division_2023_2024.Helsingør_vs_Kolding2024_03_08possession_data
Processing DNK_1_Division_2024_2025.Hvidovre_vs_Kolding2025_05_03possession_data
Processing DNK_1_Division_2023_2024.B_93_vs_Vendsyssel2023_08_26possession_data
Processing DNK_1_Division_2024_2025.Esbjerg_vs_B_932024_09_15possession_data
Processing DNK_1_Division_2023_2024.B_93_vs_SønderjyskE2023_10_27possession_data
Processing DNK_1_Division_2024_2025.HB_Køge_vs_Esbjerg2025_02_28possession_data
Processing DNK_1_Division